<a href="https://colab.research.google.com/github/dharmikyash7118/PytorchModel/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet
!pip install opendatasets --quiet

import opendatasets as od
od.download("https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dharmikyash7118
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection


100%|██████████| 3.30M/3.30M [00:00<00:00, 666MB/s]

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer,AutoModel
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device='cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
data_df=pd.read_json("/content/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json",lines=True)
data_df.dropna(inplace=True)
data_df.drop_duplicates(inplace=True)
data_df.drop(['article_link'],inplace=True,axis=1)
print(data_df.shape)
data_df.head()

(28617, 2)


,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(np.array(data_df['headline']),np.array(data_df['is_sarcastic']),test_size=0.3)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,test_size=0.5)
print(X_train.shape[0],X_val.shape[0],X_test.shape[0])

20031 4293 4293


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
class dataset(Dataset):
  def __init__(self,X,Y):
    self.X=[tokenizer(x,
                      max_length=100,
                      truncation=True,
                      padding="max_length",
                      return_tensors='pt').to(device)
                      for x in X
            ]
    self.Y=torch.tensor(Y,dtype=torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self,indx):
    return self.X[indx],self.Y[indx]

training_data=dataset(X_train,y_train)
validation_data=dataset(X_val,y_val)
testing_data=dataset(X_test,y_test)

In [ ]:
BATCH_SIZE=32
EPOCHS=10
LR=1e-4

In [ ]:
training_dataloader=DataLoader(training_data,batch_size=BATCH_SIZE)
validation_dataloader=DataLoader(validation_data,batch_size=BATCH_SIZE)
testing_dataloader=DataLoader(testing_data,batch_size=BATCH_SIZE)

In [ ]:
class MyModel(nn.Module):
  def __init__(self,bert):
    super(MyModel,self).__init__()

    self.bert=bert
    self.dropout1=nn.Dropout(0,25)
    self.linear1=nn.Linear(768,384)
    self.linear2=nn.Linear(384,1)
    self.sigmoid=nn.Sigmoid()

  def forward(self,input_ids,attention_mask):
    pooled_output=self.bert(input_ids,attention_mask,return_dict=False)[0][:,0]
    output=self.linear1(pooled_output)
    output=self.dropout1(output)
    output=self.linear2(output)
    output=self.sigmoid(output)
    return output

In [ ]:
for param in bert_model.parameters():
  param.requires_grad=False

model=MyModel(bert_model).to(device)

In [ ]:
model

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [ ]:
criterion=nn.BCELoss()
optimizer=Adam(model.parameters(),lr=LR)

In [ ]:
total_loss_train_plot=[]
total_loss_validation_plot=[]
total_acc_train_plot=[]
total_acc_validation_plot=[]

for epoch in range(EPOCHS):
  total_acc_train=0
  total_acc_val=0
  total_loss_train=0
  total_loss_val=0

  for indx,data in enumerate(training_dataloader):
    inputs,labels=data
    inputs.to(device)
    labels.to(device)
    prediction=model(inputs['input_ids'].squeeze(1),inputs['attention_mask'].squeeze(1)).squeeze(1)
    batch_loss=criterion(prediction,labels)
    total_loss_train+=batch_loss.item()

    acc=(prediction.round()==labels).sum().item()
    total_acc_train+=acc

    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  with torch.no_grad():
    for indx,data in enumerate(training_dataloader):
      inputs,labels=data
      inputs.to(device)
      labels.to(device)
      prediction=model(inputs['input_ids'].squeeze(1),inputs['attention_mask'].squeeze(1)).squeeze(1)
      batch_loss=criterion(prediction,labels)
      total_loss_val+=batch_loss.item()

      acc=(prediction.round()==labels).sum().item()
      total_acc_val+=acc

  total_loss_train_plot.append(round(total_loss_train/1000,4))
  total_loss_validation_plot.append(round(total_loss_val/1000,4))
  total_acc_train_plot.append(round(total_acc_train/(training_data.__len__())*100,4))
  total_acc_validation_plot.append(round(total_acc_val/(validation_data.__len__())*100,4))

  print(f"""
                  Epoch: {epoch+1}/{EPOCHS},
                  Train Loss: {round(total_loss_train/100, 4)},
                  Train Accuracy: {round((total_acc_train/training_data.__len__() * 100), 4)},
                  Validation Loss: {round(total_loss_val/100, 4)},
                  Validation Accuracy: {round((total_acc_val/validation_data.__len__() * 100), 4)}
                  """)
  print("-"*100)


                  Epoch: 1/10,
                  Train Loss: 1.9515,
                  Train Accuracy: 86.471,
                  Validation Loss: 1.9337,
                  Validation Accuracy: 405.1013
                  
----------------------------------------------------------------------------------------------------

                  Epoch: 2/10,
                  Train Loss: 1.9403,
                  Train Accuracy: 86.5858,
                  Validation Loss: 1.9233,
                  Validation Accuracy: 405.5439
                  
----------------------------------------------------------------------------------------------------

                  Epoch: 3/10,
                  Train Loss: 1.9299,
                  Train Accuracy: 86.6557,
                  Validation Loss: 1.9148,
                  Validation Accuracy: 405.8234
                  
----------------------------------------------------------------------------------------------------

                  Epoch: 4/1

In [ ]:
with torch.no_grad():
  total_acc_test=0
  total_loss_test=0
  for indx,data in enumerate(training_dataloader):
    inputs,labels=data
    inputs.to(device)
    labels.to(device)
    prediction=model(inputs['input_ids'].squeeze(1),inputs['attention_mask'].squeeze(1)).squeeze(1)
    batch_loss=criterion(prediction,labels)
    total_loss_test+=batch_loss.item()

    acc=(prediction.round()==labels).sum().item()
    total_acc_test+=acc

print(f"Accuracy Score on testing Data is:{(round((total_acc_train/training_data.__len__())*100,4))}")


In [ ]:
fig,axs=plt.subplots(nrows=1,ncols=2,figsize=(15,5))

axs[0].plot(total_loss_train_plot,label="Train Loss")
axs[0].plot(total_loss_validation_plot,label="Validation Loss")
axs[0].set_title("Training and Validation Loss")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].set_ylim([0,1])
axs[0].legend()

axs[1].plot(total_acc_train_plot,label="Train Accuracy")
axs[1].plot(total_acc_validation_plot,label="Validation Accuracy")
axs[1].set_title("Training and Validation Accuracy")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Accuracy")
axs[1].set_ylim([0,100])
axs[1].legend()

plt.tight_layout()
plt.show()